In [2]:
import pandas as pd
import numpy as np
import sklearn

In [3]:
df = pd.read_parquet('./data/objectnet/ground_truth/box_data.parquet')
vecs = np.load('./data/objectnet/vectors.npy')

In [4]:
import transformers
from transformers import CLIPProcessor, CLIPModel

ModuleNotFoundError: No module named 'transformers'

In [142]:
clip_model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')
clip_processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')

In [143]:
inputs = clip_processor(text=["a big banana"], return_tensors="pt", padding=True)

In [147]:
import torch
with torch.no_grad():
    query_vectors = clip_model.get_text_features(inputs['input_ids'])

In [183]:
import torchvision

In [199]:
import PIL.Image

In [208]:
def loader(x):
    return (clip_processor(images=[PIL.Image.open(x).convert('RGB')], return_tensors='pt'),
            x.split('/')[-2],
            '/'.join(x.split('/')[-2:])
    )

In [209]:

dataset = torchvision.datasets.ImageFolder('./data/objectnet/images', loader=loader)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)

output = []
for batch in dataloader:
    with torch.no_grad():
        image_vectors = clip_model.get_image_features(batch['pixel_values'])
    output.append(batch, image_vectors)
    break


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/orm/mambaforge/envs/pixeltable_39/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/orm/mambaforge/envs/pixeltable_39/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'loader' on <module '__main__' (built-in)>


In [205]:
for batch in dataloader:
    with torch.no_grad():
        image_vectors = clip_model.get_image_features(batch[0])

    break

((<PIL.Image.Image image mode=RGB size=224x224>,
  'air_freshener/01d44a4b77b44a0.png',
  'air_freshener'),
 0)

In [182]:
clip_processor(images=)
clip_model.get_image_features(a)

Signature:
clip_model.get_image_features(
    pixel_values: Optional[torch.FloatTensor] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    return_dict: Optional[bool] = None,
) -> torch.FloatTensor
Docstring:
The [`CLIPModel`] forward method, overrides the `__call__` special method.

<Tip>

Although the recipe for forward pass needs to be defined within this function, one should call the [`Module`]
instance afterwards instead of this since the former takes care of running the pre and post processing steps while
the latter silently ignores them.

</Tip>

Args:
    pixel_values (`torch.FloatTensor` of shape `(batch_size, num_channels, height, width)`):
        Pixel values. Padding will be ignored by default should you provide it. Pixel values can be obtained using
        [`AutoImageProcessor`]. See [`CLIPImageProcessor.__call__`] for details.
    output_attentions (`bool`, *optional*):
        Whether or not to return the attentio

In [145]:
df

,idx,path,category,x1,x2,y1,y2,im_width,im_height,file_path
0,0,remote_control/5ffde0ea5aca4b3.png,remote control,0,224,0,224,224,224,remote_control/5ffde0ea5aca4b3.png
1,1,remote_control/14bbfbb21fad46b.png,remote control,0,224,0,224,224,224,remote_control/14bbfbb21fad46b.png
2,2,remote_control/280ec1077fbf4af.png,remote control,0,224,0,224,224,224,remote_control/280ec1077fbf4af.png
3,3,remote_control/e9d445f6c153411.png,remote control,0,224,0,224,224,224,remote_control/e9d445f6c153411.png
4,4,remote_control/71f51802abb14f1.png,remote control,0,224,0,224,224,224,remote_control/71f51802abb14f1.png
...,...,...,...,...,...,...,...,...,...,...
50268,50268,combination_lock/3d48df5c7d7a44b.png,combination lock,0,224,0,224,224,224,combination_lock/3d48df5c7d7a44b.png
50269,50269,combination_lock/d118e9b83ef04b7.png,combination lock,0,224,0,224,224,224,combination_lock/d118e9b83ef04b7.png
50270,50270,combination_lock/5f424567488c4fc.png,combination lock,0,224,0,224,224,224,combination_lock/5f424567488c4fc.png
50271,50271,combination_lock/120a2b5010e2493.png,combination lock,0,224,0,224,224,224,combination_lock/120a2b5010e2493.png


In [148]:
scores = vecs @ query_vectors.detach().numpy().reshape(-1)

In [149]:
y_true = np.zeros(df.idx.max() + 1)

In [150]:
y_true[df[df.category == 'banana'].idx] =1

In [151]:
y_true

array([0., 0., 0., ..., 0., 0., 0.])

In [152]:
from sklearn.metrics import average_precision_score

In [166]:
qvecs = query_vectors.detach().numpy()
qvecs = qvecs / np.linalg.norm(qvecs, axis=1, keepdims=True)

In [169]:
qvec = qvecs[0]

In [172]:
scores = vecs @ qvec

In [173]:

average_precision_score(y_true, scores)

0.005697931859917126

In [178]:
np.argsort(-scores)[:10]

array([40784, 38269, 27794, 30625, 30350, 10362, 49223, 17767, 35611,
       10505])

In [181]:
#y_true[y_true]